#### 웹툰 크롤링 이미지
- 690 x 1600 크기의 image

https://sosal.kr/1067

In [1]:
import cv2
import numpy as np
import imutils
import os

In [2]:
# 한글이 포함된 파일경로에 대해 Mac에서만 동작, Windows에는 동작하지 않는 경우.
# 1. open()함수 이용 -> filepath를 인코딩한 파일을 바이트로 읽음
# 2. bytearray타입으로 바이트 읽음
# 3. bytearray를 numpy array로 convert
# cv2.imdecode -> Mac에서 imread를 통해 읽었던 결과와 같은 값 반환
def hangul_file_path(filepath):
    stream = open(filepath.encode("utf-8"), "rb")
    byte = bytearray(stream.read())
    numpy_array = np.asarray(byte, dtype=np.uint8)
    
    return cv2.imdecode(numpy_array, cv2.IMREAD_UNCHANGED)

In [3]:
# 모양을 구분해주는 Detector. shape을 반환.
class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
    # initialize the shape name and approximate the contour
        shape = "unidentified"
        # arcLength: 컨투어의 둘레
        peri = cv2.arcLength(c, True)
        # approxPolyDP: 
        # c: 컨투어 포인트 배열
        # 0.04*peri: 실제 컨투어 라인과 근사치의 최대거리
        #  -> 0.01이면 실제 컨투어 라인과 아주 근사
        #  -> 0.05면 투박
        # True: 폐곡선 여부
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        
        # 꼭지점 3개면 삼각형
        if len(approx) == 3:
            shape = "triangle"

        # 꼭지점 4개면 사각형
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            # boundingRect: 컨투어 라인을 둘러싸는 사각형을 그리는 방법 중 1개
            #  ->물체의 회전은 고려하지 않은 사각형. 고려하고 싶으면 minAreaRect
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)

            # 가로세로 비율이 1에 가까우면 정사각형, 아니면 직사각형
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

        # 꼭지점 5개면 오각형
        elif len(approx) == 5:
            shape = "pentagon"
        
        # 꼭지점 3, 4, 5 아니면 원
        else:
            shape = "circle"

        # shpae이 무엇인지 return
        return shape

In [8]:
def image_segmentation(fname):
    #image = cv2.imread(fname, 0)
    image = hangul_file_path(fname)
    # 이미지 흑백화
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 가우시안 블러
    image_gray_blurred = cv2.GaussianBlur(image_gray, (5, 5), 0)
    # threshing
    ret,thresh2 = cv2.threshold(image_gray_blurred,127,255,cv2.THRESH_BINARY_INV)
    
    sd = ShapeDetector()
    cnts = cv2.findContours(thresh2.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    
    count = 0
    for c in cnts[0]:
        if sd.detect(c) != 'rectangle': 
            next
        c = c.astype("float")
        c = c.astype("int")
        x, y, w, h = cv2.boundingRect(c)

        if not (300 < w and 300 < h): 
            #print("skip" + str(w) + " - " + str(h))
            continue
            
        new_folder = fname.split('\\')[-2]
        image_name = fname.split('\\')[-1]
        
        # 각 웹툰명에 해당하는 폴더가 없으면, 폴더 생성.
        if not os.path.exists('webtoon_result\최강자전\{}'.format(new_folder)):
            os.makedirs('webtoon_result\최강자전\{}'.format(new_folder))
            
        count = count+1
        #print("webtoon_result/네이버/{}/result_{}{}.jpg".format(new_folder, image_name[-20:-4], str(count)))
        #cv2.imwrite("webtoon_result/네이버/{}/result_{}{}.jpg".format(new_folder, fname[-20:-4], str(count)), image[y: y + h, x: x + w])
        #dst = cv2.flip(image[y: y + h, x: x + w], flipCode=1)  # 이미지 좌우 반전
        
        # opencv는 한글경로 지원되지 않음. 해결
        retval, buffer = cv2.imencode('.jpg', image[y: y + h, x: x + w])
        if (retval):
            with open("webtoon_result/최강자전/{}/result_{}{}.jpg".format(new_folder, image_name[-20:-4], str(count)), 'wb') as f_writer:
                f_writer.write(buffer)

In [9]:
for root, dirs, files in os.walk('Crawled\최강자전 웹툰'):
    for fname in files:
        full_fname = os.path.join(root, fname)
        #print(full_fname)
        image_segmentation(full_fname)